# LME銅 Cash/3Mスプレッド 包括的モデリング分析

## 分析目的
このノートブックでは、Cash/3Mスプレッドに対して以下の包括的なモデリングを実行します：

### 実装モデル:
1. **ARIMA/GARCH**: 時系列の平均・分散モデリング
2. **Random Forest**: アンサンブル機械学習
3. **XGBoost**: 勾配ブースティング
4. **LSTM**: 深層学習による時系列予測
5. **Prophet**: Facebookの時系列予測ライブラリ
6. **アンサンブル**: 複数モデルの組み合わせ

### 評価項目:
- 予測精度（MAE, RMSE, MAPE）
- 方向性精度（上昇/下降の的中率）
- リスク調整リターン（シャープレシオ）
- 取引シミュレーション

In [1]:
# ライブラリのインポート
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine
import warnings
from datetime import datetime, timedelta
import os
from dotenv import load_dotenv
from scipy import stats
import pickle
import joblib

# 機械学習ライブラリ
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import TimeSeriesSplit, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import xgboost as xgb

# 時系列分析
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from arch import arch_model

# ディープラーニング
try:
    import tensorflow as tf
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization
    from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
    from tensorflow.keras.optimizers import Adam
    TENSORFLOW_AVAILABLE = True
    print("✓ TensorFlow available")
except ImportError:
    TENSORFLOW_AVAILABLE = False
    print("TensorFlow not available - LSTM models will be skipped")

# Prophet
try:
    from prophet import Prophet
    PROPHET_AVAILABLE = True
    print("✓ Prophet available")
except ImportError:
    PROPHET_AVAILABLE = False
    print("Prophet not available - Prophet models will be skipped")

warnings.filterwarnings('ignore')
load_dotenv()

# データベース設定
db_config = {
    'host': os.getenv('DB_HOST', 'localhost'),
    'database': os.getenv('DB_NAME', 'lme_copper_db'),
    'user': os.getenv('DB_USER', 'postgres'),
    'password': os.getenv('DB_PASSWORD', 'password'),
    'port': os.getenv('DB_PORT', '5432')
}

connection_string = f"postgresql://{db_config['user']}:{db_config['password']}@{db_config['host']}:{db_config['port']}/{db_config['database']}"

# スタイル設定
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['figure.figsize'] = [15, 10]

# モデル保存ディレクトリ
model_dir = '/Users/Yusuke/claude-code/RefinitivDB/models/cash_3m_spread'
os.makedirs(model_dir, exist_ok=True)

print("✓ ライブラリ読み込み完了")
print("Ready for comprehensive Cash/3M spread modeling")

✓ TensorFlow available
✓ Prophet available
✓ ライブラリ読み込み完了
Ready for comprehensive Cash/3M spread modeling


/Users/Yusuke/claude-code/RefinitivDB/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## データ準備と特徴量エンジニアリング

In [2]:
def load_comprehensive_spread_data():
    """
    複数のデータソースからスプレッドデータを読み込み
    """
    print("データ読み込み開始...")
    
    data_sources = [
        {
            'name': 'lme_copper_prices',
            'spread_query': """
                SELECT 
                    trade_date,
                    close_price as spread_value
                FROM lme_copper_prices 
                WHERE ric_code = 'CMCU0-3'
                ORDER BY trade_date
            """,
            'component_query': """
                SELECT 
                    p1.trade_date,
                    p1.close_price as cash_price,
                    p2.close_price as future_3m_price,
                    (p1.close_price - p2.close_price) as spread_value
                FROM 
                    (SELECT trade_date, close_price FROM lme_copper_prices WHERE ric_code = 'CMCU0') p1
                INNER JOIN 
                    (SELECT trade_date, close_price FROM lme_copper_prices WHERE ric_code = 'CMCU3') p2
                    ON p1.trade_date = p2.trade_date
                ORDER BY p1.trade_date
            """
        },
        {
            'name': 'lme_copper_futures',
            'spread_query': """
                SELECT 
                    trade_date,
                    close_price as spread_value
                FROM lme_copper_futures 
                WHERE ric_code = 'CMCU0-3'
                ORDER BY trade_date
            """,
            'component_query': """
                SELECT 
                    p1.trade_date,
                    p1.close_price as cash_price,
                    p2.close_price as future_3m_price,
                    (p1.close_price - p2.close_price) as spread_value
                FROM 
                    (SELECT trade_date, close_price FROM lme_copper_futures WHERE ric_code = 'CMCU0') p1
                INNER JOIN 
                    (SELECT trade_date, close_price FROM lme_copper_futures WHERE ric_code = 'CMCU3') p2
                    ON p1.trade_date = p2.trade_date
                ORDER BY p1.trade_date
            """
        }
    ]
    
    try:
        engine = create_engine(connection_string)
        
        for source in data_sources:
            print(f"\n🔍 {source['name']}からデータ取得中...")
            
            # 直接スプレッドデータを試行
            try:
                df = pd.read_sql(source['spread_query'], engine)
                if df is not None and len(df) > 0:
                    df['trade_date'] = pd.to_datetime(df['trade_date'])
                    df.set_index('trade_date', inplace=True)
                    print(f"✅ 直接スプレッドデータ: {len(df)} レコード")
                    return df
            except Exception as e:
                print(f"   直接スプレッドデータ取得失敗: {e}")
            
            # コンポーネントから計算を試行
            try:
                df = pd.read_sql(source['component_query'], engine)
                if df is not None and len(df) > 0:
                    df['trade_date'] = pd.to_datetime(df['trade_date'])
                    df.set_index('trade_date', inplace=True)
                    print(f"✅ コンポーネント計算: {len(df)} レコード")
                    return df
            except Exception as e:
                print(f"   コンポーネント計算失敗: {e}")
    
    except Exception as e:
        print(f"⚠️ データベース接続失敗: {e}")
    
    # フォールバック: ダミーデータ生成
    print("\n🎲 ダミーデータ生成中...")
    
    date_range = pd.date_range(
        start='2020-01-01', 
        end='2024-12-31', 
        freq='D'
    )
    
    # 営業日のみにフィルタ（土日除外）
    business_days = date_range[date_range.dayofweek < 5]
    
    np.random.seed(42)
    
    # 現実的なスプレッドデータ生成（ARプロセス + ボラティリティクラスター）
    n = len(business_days)
    base_spread = 25
    ar_coeff = 0.95  # 強い持続性
    volatility_base = 15
    
    # ARプロセス
    spread_values = np.zeros(n)
    spread_values[0] = base_spread
    
    # ボラティリティクラスター
    volatility = np.random.lognormal(mean=np.log(volatility_base), sigma=0.3, size=n)
    
    for i in range(1, n):
        trend = np.sin(2 * np.pi * i / 252) * 5  # 年次季節性
        noise = np.random.normal(0, volatility[i])
        spread_values[i] = ar_coeff * spread_values[i-1] + (1-ar_coeff) * (base_spread + trend) + noise
    
    df = pd.DataFrame({
        'spread_value': spread_values
    }, index=business_days)
    
    df.index.name = 'trade_date'
    
    print(f"✅ 高度なダミーデータ生成完了: {len(df)} レコード")
    print(f"期間: {df.index.min()} ～ {df.index.max()}")
    print(f"スプレッド範囲: {df['spread_value'].min():.2f} ～ {df['spread_value'].max():.2f}")
    
    return df

# データ読み込み
df = load_comprehensive_spread_data()
print(f"\n📊 最終データセット: {len(df)} レコード")
print(df.head())

データ読み込み開始...

🔍 lme_copper_pricesからデータ取得中...
   直接スプレッドデータ取得失敗: (psycopg2.errors.UndefinedColumn) column "close_price" does not exist
LINE 4:                     close_price as spread_value
                            ^
HINT:  Perhaps you meant to reference the column "lme_copper_prices.last_price" or the column "lme_copper_prices.low_price".

[SQL: 
                SELECT 
                    trade_date,
                    close_price as spread_value
                FROM lme_copper_prices 
                WHERE ric_code = 'CMCU0-3'
                ORDER BY trade_date
            ]
(Background on this error at: https://sqlalche.me/e/20/f405)
   コンポーネント計算失敗: (psycopg2.errors.UndefinedColumn) column "close_price" does not exist
LINE 8:                     (SELECT trade_date, close_price FROM lme...
                                                ^
HINT:  Perhaps you meant to reference the column "lme_copper_prices.last_price" or the column "lme_copper_prices.low_price".

[SQL: 
        

## モデル1: ARIMA + GARCH

In [3]:
# ARIMA + GARCH モデル
class ARIMAGARCHModel:
    def __init__(self):
        self.arima_model = None
        self.garch_model = None
        self.fitted_arima = None
        self.fitted_garch = None
        
    def fit(self, data, arima_order=(1,1,1), garch_p=1, garch_q=1):
        """ARIMA + GARCH モデルの学習"""
        try:
            # ARIMA フィッティング
            self.arima_model = ARIMA(data, order=arima_order)
            self.fitted_arima = self.arima_model.fit()
            
            # ARIMA残差を取得
            residuals = self.fitted_arima.resid
            
            # GARCH フィッティング（残差の分散モデリング）
            self.garch_model = arch_model(residuals, vol='GARCH', p=garch_p, q=garch_q)
            self.fitted_garch = self.garch_model.fit(disp='off')
            
            return True
        except Exception as e:
            print(f"ARIMA-GARCH フィッティングエラー: {e}")
            return False
    
    def predict(self, steps=1):
        """予測実行"""
        if self.fitted_arima is None:
            return None, None
        
        # ARIMA予測
        arima_forecast = self.fitted_arima.forecast(steps=steps)
        
        # GARCH予測（ボラティリティ）
        if self.fitted_garch is not None:
            garch_forecast = self.fitted_garch.forecast(horizon=steps)
            volatility = np.sqrt(garch_forecast.variance.iloc[-1, :].values)
        else:
            volatility = None
        
        return arima_forecast, volatility

# ARIMA-GARCH実行
if data is not None:
    print("ARIMA + GARCH モデル学習中...")
    
    # データ分割
    train_size = int(len(data) * 0.8)
    train_data = data['spread_price'][:train_size]
    test_data = data['spread_price'][train_size:]
    
    # モデル学習
    arima_garch = ARIMAGARCHModel()
    success = arima_garch.fit(train_data)
    
    if success:
        print("✅ ARIMA-GARCH学習完了")
        
        # テストデータ予測
        predictions_ag = []
        volatilities_ag = []
        
        # ウォークフォワード予測
        for i in range(len(test_data)):
            pred, vol = arima_garch.predict(steps=1)
            predictions_ag.append(pred.iloc[0] if hasattr(pred, 'iloc') else pred[0])
            volatilities_ag.append(vol[0] if vol is not None else np.nan)
            
            # モデル更新（新しいデータポイントを追加）
            if i < len(test_data) - 1:
                updated_data = pd.concat([train_data, test_data.iloc[:i+1]])
                arima_garch.fit(updated_data)
        
        predictions_ag = pd.Series(predictions_ag, index=test_data.index)
        
        # 評価指標計算
        mae_ag = mean_absolute_error(test_data, predictions_ag)
        rmse_ag = np.sqrt(mean_squared_error(test_data, predictions_ag))
        
        print(f"ARIMA-GARCH MAE: {mae_ag:.3f}")
        print(f"ARIMA-GARCH RMSE: {rmse_ag:.3f}")
        
        # モデル保存
        with open(f'{model_dir}/arima_garch_model.pkl', 'wb') as f:
            pickle.dump(arima_garch, f)
        
    else:
        predictions_ag = None
        print("❌ ARIMA-GARCH学習失敗")

NameError: name 'data' is not defined

## モデル2: Random Forest

In [ ]:
# Random Forest モデル
def prepare_ml_data(data, target_col='spread_price', forecast_horizon=1):
    """機械学習用データ準備"""
    # 特徴量選択（基本的な数値特徴量のみ）
    feature_cols = [col for col in data.columns if col not in [target_col] 
                   and not col.startswith('spread_lag')  # ラグ特徴量は別途処理
                   and data[col].dtype in ['int64', 'float64']]
    
    # ラグ特徴量を追加（データリークを防ぐ）
    lag_features = [col for col in data.columns if col.startswith('spread_lag')]
    feature_cols.extend(lag_features)
    
    X = data[feature_cols].copy()
    
    # ターゲット変数（将来の値を予測）
    y = data[target_col].shift(-forecast_horizon)  # 1期先を予測
    
    # 欠損値処理
    valid_idx = ~(X.isna().any(axis=1) | y.isna())
    X = X[valid_idx]
    y = y[valid_idx]
    
    return X, y, feature_cols

if data is not None:
    print("Random Forest モデル学習中...")
    
    # データ準備
    X, y, feature_cols = prepare_ml_data(data)
    
    print(f"特徴量数: {len(feature_cols)}")
    print(f"サンプル数: {len(X)}")
    
    # データ分割
    train_size = int(len(X) * 0.8)
    X_train, X_test = X[:train_size], X[train_size:]
    y_train, y_test = y[:train_size], y[train_size:]
    
    # 特徴量スケーリング
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Random Forest ハイパーパラメータ調整
    rf_params = {
        'n_estimators': [100, 200, 300],
        'max_depth': [10, 20, None],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    }
    
    # Time Series Cross Validation
    tscv = TimeSeriesSplit(n_splits=5)
    
    rf = RandomForestRegressor(random_state=42, n_jobs=-1)
    rf_search = RandomizedSearchCV(
        rf, rf_params, n_iter=20, cv=tscv, 
        scoring='neg_mean_absolute_error', n_jobs=-1, random_state=42
    )
    
    rf_search.fit(X_train_scaled, y_train)
    best_rf = rf_search.best_estimator_
    
    print(f"最適パラメータ: {rf_search.best_params_}")
    
    # 予測
    predictions_rf = best_rf.predict(X_test_scaled)
    predictions_rf = pd.Series(predictions_rf, index=y_test.index)
    
    # 評価
    mae_rf = mean_absolute_error(y_test, predictions_rf)
    rmse_rf = np.sqrt(mean_squared_error(y_test, predictions_rf))
    r2_rf = r2_score(y_test, predictions_rf)
    
    print(f"Random Forest MAE: {mae_rf:.3f}")
    print(f"Random Forest RMSE: {rmse_rf:.3f}")
    print(f"Random Forest R²: {r2_rf:.3f}")
    
    # 特徴量重要度
    feature_importance = pd.DataFrame({
        'feature': feature_cols,
        'importance': best_rf.feature_importances_
    }).sort_values('importance', ascending=False)
    
    print("\n上位10特徴量:")
    print(feature_importance.head(10))
    
    # モデル保存
    joblib.dump(best_rf, f'{model_dir}/random_forest_model.pkl')
    joblib.dump(scaler, f'{model_dir}/scaler.pkl')
    
    print("✅ Random Forest学習完了")

## モデル3: XGBoost

In [ ]:
# XGBoost モデル
if data is not None:
    print("XGBoost モデル学習中...")
    
    # XGBoost ハイパーパラメータ
    xgb_params = {
        'n_estimators': [100, 200, 300],
        'max_depth': [3, 6, 9],
        'learning_rate': [0.01, 0.1, 0.2],
        'subsample': [0.8, 0.9, 1.0],
        'colsample_bytree': [0.8, 0.9, 1.0]
    }
    
    xgb_model = xgb.XGBRegressor(random_state=42, n_jobs=-1)
    xgb_search = RandomizedSearchCV(
        xgb_model, xgb_params, n_iter=20, cv=tscv,
        scoring='neg_mean_absolute_error', n_jobs=-1, random_state=42
    )
    
    xgb_search.fit(X_train_scaled, y_train)
    best_xgb = xgb_search.best_estimator_
    
    print(f"最適パラメータ: {xgb_search.best_params_}")
    
    # 予測
    predictions_xgb = best_xgb.predict(X_test_scaled)
    predictions_xgb = pd.Series(predictions_xgb, index=y_test.index)
    
    # 評価
    mae_xgb = mean_absolute_error(y_test, predictions_xgb)
    rmse_xgb = np.sqrt(mean_squared_error(y_test, predictions_xgb))
    r2_xgb = r2_score(y_test, predictions_xgb)
    
    print(f"XGBoost MAE: {mae_xgb:.3f}")
    print(f"XGBoost RMSE: {rmse_xgb:.3f}")
    print(f"XGBoost R²: {r2_xgb:.3f}")
    
    # 特徴量重要度
    xgb_importance = pd.DataFrame({
        'feature': feature_cols,
        'importance': best_xgb.feature_importances_
    }).sort_values('importance', ascending=False)
    
    print("\n上位10特徴量:")
    print(xgb_importance.head(10))
    
    # モデル保存
    joblib.dump(best_xgb, f'{model_dir}/xgboost_model.pkl')
    
    print("✅ XGBoost学習完了")

## モデル4: LSTM （深層学習）

In [ ]:
# LSTM モデル
def create_lstm_sequences(data, sequence_length=30):
    """LSTM用のシーケンスデータ作成"""
    sequences = []
    targets = []
    
    for i in range(sequence_length, len(data)):
        sequences.append(data[i-sequence_length:i])
        targets.append(data[i])
    
    return np.array(sequences), np.array(targets)

if TENSORFLOW_AVAILABLE and data is not None:
    print("LSTM モデル学習中...")
    
    # LSTM用データ準備
    spread_values = data['spread_price'].values
    
    # 正規化
    lstm_scaler = MinMaxScaler()
    spread_scaled = lstm_scaler.fit_transform(spread_values.reshape(-1, 1)).flatten()
    
    # シーケンス作成
    sequence_length = 30
    X_seq, y_seq = create_lstm_sequences(spread_scaled, sequence_length)
    
    # データ分割
    train_size = int(len(X_seq) * 0.8)
    X_train_seq = X_seq[:train_size]
    X_test_seq = X_seq[train_size:]
    y_train_seq = y_seq[:train_size]
    y_test_seq = y_seq[train_size:]
    
    # LSTM形状調整
    X_train_seq = X_train_seq.reshape((X_train_seq.shape[0], X_train_seq.shape[1], 1))
    X_test_seq = X_test_seq.reshape((X_test_seq.shape[0], X_test_seq.shape[1], 1))
    
    # LSTMモデル構築
    model_lstm = Sequential([
        LSTM(50, return_sequences=True, input_shape=(sequence_length, 1)),
        Dropout(0.2),
        LSTM(50, return_sequences=False),
        Dropout(0.2),
        Dense(25),
        Dense(1)
    ])
    
    model_lstm.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])
    
    # コールバック
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001)
    
    # 学習
    history = model_lstm.fit(
        X_train_seq, y_train_seq,
        batch_size=32,
        epochs=100,
        validation_split=0.2,
        callbacks=[early_stopping, reduce_lr],
        verbose=0
    )
    
    # 予測
    predictions_lstm_scaled = model_lstm.predict(X_test_seq, verbose=0)
    predictions_lstm = lstm_scaler.inverse_transform(predictions_lstm_scaled).flatten()
    
    # 実際の値も逆変換
    y_test_actual = lstm_scaler.inverse_transform(y_test_seq.reshape(-1, 1)).flatten()
    
    # 評価
    mae_lstm = mean_absolute_error(y_test_actual, predictions_lstm)
    rmse_lstm = np.sqrt(mean_squared_error(y_test_actual, predictions_lstm))
    
    print(f"LSTM MAE: {mae_lstm:.3f}")
    print(f"LSTM RMSE: {rmse_lstm:.3f}")
    
    # 予測結果をSeriesに変換（インデックス調整）
    test_dates = data.index[train_size + sequence_length:]
    predictions_lstm = pd.Series(predictions_lstm, index=test_dates[:len(predictions_lstm)])
    
    # モデル保存
    model_lstm.save(f'{model_dir}/lstm_model.h5')
    joblib.dump(lstm_scaler, f'{model_dir}/lstm_scaler.pkl')
    
    print("✅ LSTM学習完了")
    
else:
    predictions_lstm = None
    if not TENSORFLOW_AVAILABLE:
        print("⏭️ TensorFlowが利用できないため、LSTMをスキップ")

## モデル5: Prophet

In [ ]:
# Prophet モデル
if PROPHET_AVAILABLE and data is not None:
    print("Prophet モデル学習中...")
    
    # Prophet用データ準備
    prophet_data = pd.DataFrame({
        'ds': data.index,
        'y': data['spread_price'].values
    })
    
    # データ分割
    train_size = int(len(prophet_data) * 0.8)
    prophet_train = prophet_data[:train_size]
    prophet_test = prophet_data[train_size:]
    
    # Prophetモデル
    model_prophet = Prophet(
        daily_seasonality=False,
        weekly_seasonality=True,
        yearly_seasonality=True,
        changepoint_prior_scale=0.05,
        seasonality_prior_scale=10.0
    )
    
    # カスタム季節性追加
    model_prophet.add_seasonality(name='monthly', period=30.5, fourier_order=5)
    model_prophet.add_seasonality(name='quarterly', period=91.25, fourier_order=8)
    
    # 学習
    model_prophet.fit(prophet_train)
    
    # 予測
    future_prophet = model_prophet.make_future_dataframe(periods=len(prophet_test), freq='D')
    forecast_prophet = model_prophet.predict(future_prophet)
    
    # テスト期間の予測抽出
    predictions_prophet = forecast_prophet['yhat'].iloc[train_size:].values
    predictions_prophet = pd.Series(predictions_prophet, index=prophet_test['ds'])
    
    # 評価
    mae_prophet = mean_absolute_error(prophet_test['y'], predictions_prophet)
    rmse_prophet = np.sqrt(mean_squared_error(prophet_test['y'], predictions_prophet))
    
    print(f"Prophet MAE: {mae_prophet:.3f}")
    print(f"Prophet RMSE: {rmse_prophet:.3f}")
    
    # モデル保存
    with open(f'{model_dir}/prophet_model.pkl', 'wb') as f:
        pickle.dump(model_prophet, f)
    
    print("✅ Prophet学習完了")
    
else:
    predictions_prophet = None
    if not PROPHET_AVAILABLE:
        print("⏭️ Prophetが利用できないため、Prophetをスキップ")

## モデル6: アンサンブル予測

In [ ]:
# アンサンブル予測
def create_ensemble_predictions(*predictions_list):
    """複数の予測結果をアンサンブル"""
    # 有効な予測のみ使用
    valid_predictions = [pred for pred in predictions_list if pred is not None]
    
    if not valid_predictions:
        return None
    
    # 共通のインデックスを見つける
    common_index = valid_predictions[0].index
    for pred in valid_predictions[1:]:
        common_index = common_index.intersection(pred.index)
    
    # 各予測を共通インデックスに合わせる
    aligned_predictions = []
    for pred in valid_predictions:
        aligned_predictions.append(pred.loc[common_index])
    
    # 単純平均でアンサンブル
    ensemble_pred = pd.concat(aligned_predictions, axis=1).mean(axis=1)
    
    return ensemble_pred

if data is not None:
    print("アンサンブル予測作成中...")
    
    # 利用可能な予測を収集
    available_predictions = []
    model_names = []
    
    if 'predictions_rf' in locals():
        available_predictions.append(predictions_rf)
        model_names.append('Random Forest')
    
    if 'predictions_xgb' in locals():
        available_predictions.append(predictions_xgb)
        model_names.append('XGBoost')
    
    if predictions_lstm is not None:
        available_predictions.append(predictions_lstm)
        model_names.append('LSTM')
    
    if predictions_prophet is not None:
        available_predictions.append(predictions_prophet)
        model_names.append('Prophet')
    
    if available_predictions:
        ensemble_predictions = create_ensemble_predictions(*available_predictions)
        
        # アンサンブル評価（共通インデックスの実際値と比較）
        common_test_data = y_test.loc[ensemble_predictions.index]
        
        mae_ensemble = mean_absolute_error(common_test_data, ensemble_predictions)
        rmse_ensemble = np.sqrt(mean_squared_error(common_test_data, ensemble_predictions))
        
        print(f"アンサンブル使用モデル: {', '.join(model_names)}")
        print(f"アンサンブル MAE: {mae_ensemble:.3f}")
        print(f"アンサンブル RMSE: {rmse_ensemble:.3f}")
        
        print("✅ アンサンブル予測完了")
    else:
        ensemble_predictions = None
        print("❌ アンサンブル用の予測が不足")

## モデル比較と可視化

In [ ]:
# 全モデルの結果比較
if data is not None:
    print("\n" + "="*60)
    print("全モデル パフォーマンス比較")
    print("="*60)
    
    # 結果まとめ
    results = []
    
    if 'mae_ag' in locals() and predictions_ag is not None:
        results.append(['ARIMA-GARCH', mae_ag, rmse_ag])
    
    if 'mae_rf' in locals():
        results.append(['Random Forest', mae_rf, rmse_rf])
    
    if 'mae_xgb' in locals():
        results.append(['XGBoost', mae_xgb, rmse_xgb])
    
    if 'mae_lstm' in locals():
        results.append(['LSTM', mae_lstm, rmse_lstm])
    
    if 'mae_prophet' in locals():
        results.append(['Prophet', mae_prophet, rmse_prophet])
    
    if 'mae_ensemble' in locals():
        results.append(['Ensemble', mae_ensemble, rmse_ensemble])
    
    # 結果表示
    results_df = pd.DataFrame(results, columns=['Model', 'MAE', 'RMSE'])
    results_df = results_df.sort_values('MAE')
    
    print(results_df.to_string(index=False))
    
    # 最良モデル
    best_model = results_df.iloc[0]['Model']
    best_mae = results_df.iloc[0]['MAE']
    
    print(f"\n🏆 最良モデル: {best_model} (MAE: {best_mae:.3f})")
    
    # 予測結果可視化
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(20, 15))
    
    # 実際の値（テストデータ期間）
    test_period_data = data['spread_price'][train_size:]
    ax1.plot(test_period_data.index, test_period_data.values, 
             label='実際の値', linewidth=2, color='black', alpha=0.8)
    
    # 各モデルの予測
    colors = ['red', 'blue', 'green', 'orange', 'purple', 'brown']
    
    if 'predictions_rf' in locals():
        ax1.plot(predictions_rf.index, predictions_rf.values, 
                 label='Random Forest', linestyle='--', color=colors[0], alpha=0.7)
    
    if 'predictions_xgb' in locals():
        ax1.plot(predictions_xgb.index, predictions_xgb.values, 
                 label='XGBoost', linestyle='--', color=colors[1], alpha=0.7)
    
    if predictions_lstm is not None:
        ax1.plot(predictions_lstm.index, predictions_lstm.values, 
                 label='LSTM', linestyle='--', color=colors[2], alpha=0.7)
    
    if predictions_prophet is not None:
        ax1.plot(predictions_prophet.index, predictions_prophet.values, 
                 label='Prophet', linestyle='--', color=colors[3], alpha=0.7)
    
    if ensemble_predictions is not None:
        ax1.plot(ensemble_predictions.index, ensemble_predictions.values, 
                 label='Ensemble', linestyle='-', color=colors[4], linewidth=2, alpha=0.9)
    
    ax1.axhline(y=0, color='gray', linestyle='-', alpha=0.5)
    ax1.set_title('Cash/3M Spread 予測結果比較', fontsize=16, fontweight='bold')
    ax1.set_ylabel('スプレッド (USD/tonne)')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # MAE比較バーチャート
    ax2.bar(results_df['Model'], results_df['MAE'], color=colors[:len(results_df)], alpha=0.7)
    ax2.set_title('モデル別 MAE 比較', fontsize=16, fontweight='bold')
    ax2.set_ylabel('MAE')
    ax2.tick_params(axis='x', rotation=45)
    ax2.grid(True, alpha=0.3)
    
    # 値ラベル追加
    for i, v in enumerate(results_df['MAE']):
        ax2.text(i, v + 0.01, f'{v:.3f}', ha='center', va='bottom', fontweight='bold')
    
    plt.tight_layout()
    plt.savefig('../generated_images/cash_3m_spread_model_comparison.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    # 方向性精度計算
    print("\n" + "="*60)
    print("方向性精度 (上昇/下降の的中率)")
    print("="*60)
    
    actual_direction = np.sign(test_period_data.diff().dropna())
    
    direction_results = []
    
    if 'predictions_rf' in locals():
        pred_direction = np.sign(predictions_rf.diff().dropna())
        common_idx = actual_direction.index.intersection(pred_direction.index)
        direction_acc = (actual_direction.loc[common_idx] == pred_direction.loc[common_idx]).mean()
        direction_results.append(['Random Forest', direction_acc])
    
    if 'predictions_xgb' in locals():
        pred_direction = np.sign(predictions_xgb.diff().dropna())
        common_idx = actual_direction.index.intersection(pred_direction.index)
        direction_acc = (actual_direction.loc[common_idx] == pred_direction.loc[common_idx]).mean()
        direction_results.append(['XGBoost', direction_acc])
    
    if ensemble_predictions is not None:
        pred_direction = np.sign(ensemble_predictions.diff().dropna())
        common_idx = actual_direction.index.intersection(pred_direction.index)
        direction_acc = (actual_direction.loc[common_idx] == pred_direction.loc[common_idx]).mean()
        direction_results.append(['Ensemble', direction_acc])
    
    direction_df = pd.DataFrame(direction_results, columns=['Model', 'Direction_Accuracy'])
    direction_df['Direction_Accuracy_Pct'] = direction_df['Direction_Accuracy'] * 100
    direction_df = direction_df.sort_values('Direction_Accuracy', ascending=False)
    
    print(direction_df[['Model', 'Direction_Accuracy_Pct']].to_string(index=False))
    
    print(f"\n{'='*60}")
    print("Cash/3M スプレッド モデリング完了")
    print(f"{'='*60}")

## 取引シミュレーション

In [ ]:
# 簡単な取引シミュレーション
def run_trading_simulation(predictions, actual_prices, model_name="Model"):
    """予測に基づく取引シミュレーション"""
    if predictions is None or len(predictions) == 0:
        return None
    
    # 共通インデックス
    common_idx = predictions.index.intersection(actual_prices.index)
    pred_aligned = predictions.loc[common_idx]
    actual_aligned = actual_prices.loc[common_idx]
    
    if len(pred_aligned) < 2:
        return None
    
    # 取引シグナル生成
    predicted_changes = pred_aligned.diff()
    actual_changes = actual_aligned.diff()
    
    # シグナル: 予測上昇なら買い(1)、予測下降なら売り(-1)
    signals = np.where(predicted_changes > 0, 1, -1)
    signals = pd.Series(signals, index=predicted_changes.index)
    
    # リターン計算（実際の価格変化 × シグナル）
    strategy_returns = actual_changes * signals.shift(1)  # 1期前のシグナルで取引
    strategy_returns = strategy_returns.dropna()
    
    # パフォーマンス計算
    total_return = strategy_returns.sum()
    win_rate = (strategy_returns > 0).mean()
    avg_win = strategy_returns[strategy_returns > 0].mean() if (strategy_returns > 0).any() else 0
    avg_loss = strategy_returns[strategy_returns < 0].mean() if (strategy_returns < 0).any() else 0
    sharpe_ratio = strategy_returns.mean() / strategy_returns.std() if strategy_returns.std() > 0 else 0
    
    return {
        'model': model_name,
        'total_return': total_return,
        'win_rate': win_rate,
        'avg_win': avg_win,
        'avg_loss': avg_loss,
        'sharpe_ratio': sharpe_ratio,
        'num_trades': len(strategy_returns)
    }

if data is not None:
    print("\n" + "="*60)
    print("取引シミュレーション結果")
    print("="*60)
    
    simulation_results = []
    
    # 各モデルでシミュレーション実行
    if 'predictions_rf' in locals():
        sim_rf = run_trading_simulation(predictions_rf, test_period_data, "Random Forest")
        if sim_rf:
            simulation_results.append(sim_rf)
    
    if 'predictions_xgb' in locals():
        sim_xgb = run_trading_simulation(predictions_xgb, test_period_data, "XGBoost")
        if sim_xgb:
            simulation_results.append(sim_xgb)
    
    if ensemble_predictions is not None:
        sim_ensemble = run_trading_simulation(ensemble_predictions, test_period_data, "Ensemble")
        if sim_ensemble:
            simulation_results.append(sim_ensemble)
    
    # Buy & Hold 戦略（ベンチマーク）
    buy_hold_return = test_period_data.iloc[-1] - test_period_data.iloc[0]
    
    # 結果表示
    if simulation_results:
        sim_df = pd.DataFrame(simulation_results)
        sim_df['win_rate_pct'] = sim_df['win_rate'] * 100
        
        print("モデル別取引パフォーマンス:")
        display_cols = ['model', 'total_return', 'win_rate_pct', 'sharpe_ratio', 'num_trades']
        print(sim_df[display_cols].to_string(index=False))
        
        print(f"\nベンチマーク (Buy & Hold): {buy_hold_return:.2f}")
        
        # 最良取引戦略
        best_strategy = sim_df.loc[sim_df['total_return'].idxmax()]
        print(f"\n🏆 最良取引戦略: {best_strategy['model']}")
        print(f"   総リターン: {best_strategy['total_return']:.2f}")
        print(f"   勝率: {best_strategy['win_rate_pct']:.1f}%")
        print(f"   シャープレシオ: {best_strategy['sharpe_ratio']:.3f}")
    
    print(f"\n{'='*60}")
    print("分析完了: Cash/3Mスプレッドの包括的モデリング")
    print(f"{'='*60}")